# BEFORE YOU BEGIN

This code requires you to get access to Meta's gated repo at https://huggingface.co/meta-llama/Meta-Llama-3.1-8B, you will then need to create a "write" token within hugging face to ensure your abilty to download the file. Follow the instructions at https://huggingface.co/docs/hub/en/security-tokens if you are unsure of how to proceed. 

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import mlflow

from huggingface_hub import login
token = "hf_..."
login(token=token)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/ubuntu/.cache/huggingface/token
Login successful


In [2]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
output_dir = "/mnt/artifacts/llama3_1-results"
new_model = "llama3_1_8b_miniguanaco"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: NVIDIA A10G. Max memory: 22.191 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
# Load dataset
dataset_name = "mlabonne/guanaco-llama2-1k"
dataset = load_dataset(dataset_name, split="train")


In [5]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 52,
        output_dir = output_dir,
        report_to=["mlflow"]
    ),
)

/home/ubuntu/.local/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/ubuntu/.local/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/ubuntu/.local/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [6]:
experiment_name = 'llama3_1-8b-4bit-lora-ft'
exp = mlflow.set_experiment(experiment_name)
print("Fine-tuning model:")
with mlflow.start_run() as run:
    # Start training
    trainer.train()
    # Save trained model
    trainer.model.save_pretrained(f"{output_dir}/{new_model}")
FastLanguageModel.for_inference(model)

Fine-tuning model:


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.698400
2,1.389700
3,1.597600
4,1.317700
5,1.563100
6,1.433200
7,1.565500
8,1.421700
9,1.433900
10,1.752500


In [7]:
prompt = "What should i do in Paris?"

text = f"<s>[INST] {prompt} [/INST]"


inputs = tokenizer(text, return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 200, use_cache = True)
llm_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
cleaned_output = llm_output.replace(f"[INST] {prompt} [/INST]", '')
print(cleaned_output)

<s> Paris is a city of many attractions and it can be difficult to know where to start. Here are some suggestions for things to do in Paris:

1. Visit the Eiffel Tower: The Eiffel Tower is one of the most iconic landmarks in Paris and is a must-see for any visitor. You can take the elevator to the top for a stunning view of the city.
2. Visit the Louvre Museum: The Louvre is one of the largest and most famous art museums in the world, and is home to many famous works of art, including the Mona Lisa.
3. Take a stroll along the Seine River: The Seine River is a beautiful and iconic feature of Paris, and is a great place to take a leisurely stroll or take a boat ride.
4. Visit Notre Dame Cathedral: Notre Dame Cathedral is a stunning example of Gothic architecture and is one of the most famous churches in the world.
5. Explore the Montmartre neighborhood: Montmartre
